In [57]:
#Import lib
from turtle import pd
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta


In [58]:
#Load Data
portfolio = ['GEV', 'TGTX', 'PRVA', 'CEVA', 'GILD', 'KD', 'NRG']
bench_data = ['^GSPC', '^DJI', 'IWM','SPY']

end_date = datetime.now()
start_date = end_date - \
            timedelta(days=3650) # 10 years of data

portfolio_data = yf.download(portfolio, start = start_date, end = end_date) ['Close']
bench_data = yf.download (bench_data, start = start_date, end = end_date) ['Close']    

data = pd.concat([portfolio_data, bench_data], axis=1).dropna()
returns = data.pct_change().dropna()


[*********************100%***********************]  7 of 7 completed
[*********************100%***********************]  4 of 4 completed


In [59]:
# Define windows
vol_window = 63       # ~3 months trading days for volatility
beta_window = 252     # ~12 months trading days for beta
drawdown_window = 252 # 1 year for drawdown
total_return_window = len(data)  # full length (10 years approx)


In [60]:
# Initialize output DataFrame
table = pd.DataFrame(index=portfolio)

In [61]:
# Ticker
table['Ticker'] = portfolio


In [62]:
# Portfolio Weight (equal weighted)
table['Portfolio Weight'] = 1 / len(portfolio)

In [63]:
# Annualized Volatility (trailing 3 months)
table['Annualized Volatility'] = returns[portfolio].iloc[-vol_window:].std() * np.sqrt(252)


In [64]:
# Beta against SPY, IWM, DIA (trailing 12 months)
for bench in bench_data:
    bench_ret = returns[bench].iloc[-beta_window:]
    beta_vals = []
    for asset in portfolio:
        asset_ret = returns[asset].iloc[-beta_window:]
        cov = np.cov(asset_ret, bench_ret)[0, 1]
        var = np.var(bench_ret)
        beta_vals.append(cov / var)
    table[f'Beta_{bench}'] = beta_vals


In [65]:
# Average and Max Weekly Drawdown (52-week low - 52-week high) / 52-week high
weekly_high = data[portfolio].rolling(window=drawdown_window).max()
weekly_low = data[portfolio].rolling(window=drawdown_window).min()

weekly_drawdown = (weekly_low - weekly_high) / weekly_high

table['Avg Weekly Drawdown'] = weekly_drawdown.mean()
table['Max Weekly Drawdown'] = weekly_drawdown.min()


In [66]:
# Total Return (10 years)
table['Total Return'] = (data[portfolio].iloc[-1] / data[portfolio].iloc[0]) - 1


In [67]:
# Annualized Total Return
table['Annualized Total Return'] = (1 + table['Total Return']) ** (1/10) - 1

In [68]:
# Reset index to have Ticker as a column 
table.reset_index(drop=True, inplace=True)

print(table)

  Ticker  Portfolio Weight  Annualized Volatility  Beta_IWM  Beta_SPY  \
0    GEV          0.142857               0.658332  1.329231  1.780732   
1   TGTX          0.142857               0.675711  0.884579  0.873216   
2   PRVA          0.142857               0.374317  0.823643  0.638686   
3   CEVA          0.142857               0.921195  1.539098  1.940341   
4   GILD          0.142857               0.326839  0.261733  0.295724   
5     KD          0.142857               0.539235  1.128769  1.320774   
6    NRG          0.142857               0.823940  1.212271  1.656051   

   Beta_^DJI  Beta_^GSPC  Avg Weekly Drawdown  Max Weekly Drawdown  \
0   1.655671    1.861067            -0.671393            -0.719888   
1   1.091930    0.943273            -0.675826            -0.707317   
2   0.938776    0.676384            -0.373665            -0.373686   
3   1.860946    1.994881            -0.580385            -0.580385   
4   0.536989    0.299582            -0.475650            -0.47565

In [69]:

# Define portfolio and ETFs
portfolio_tickers = ['GEV', 'TGTX', 'PRVA', 'CEVA', 'GILD', 'KD', 'NRG']
etf_tickers = ['SPY', 'IWM', 'DIA']

# Dates
end_date = datetime.now()
start_date = end_date - timedelta(days=365*10)

# Download historical data
data_portfolio = yf.download(portfolio_tickers, start=start_date, end=end_date)['Close']
data_etfs = yf.download(etf_tickers, start=start_date, end=end_date)['Close']

# Calculate daily returns
returns_portfolio = data_portfolio.pct_change().dropna()
returns_etfs = data_etfs.pct_change().dropna()

# Equally weighted portfolio returns
weights = np.repeat(1 / len(portfolio_tickers), len(portfolio_tickers))
portfolio_returns = returns_portfolio.dot(weights)

# Assume current risk-free rate (annual), e.g. 5% = 0.05
risk_free_rate = 0.05

# Number of trading days in a year
trading_days = 252

# Initialize dataframe to store results
comparison_table = pd.DataFrame(index=etf_tickers)
comparison_table.index.name = 'ETF Ticker'

for etf in etf_tickers:
    etf_ret = returns_etfs[etf]

    # Correlation portfolio vs ETF
    corr = portfolio_returns.corr(etf_ret)

    # Covariance portfolio vs ETF
    cov = portfolio_returns.cov(etf_ret)

    # Tracking error = std dev of (portfolio return - ETF return)
    tracking_error = (portfolio_returns - etf_ret).std() * np.sqrt(trading_days)

    # Sharpe Ratio = (Annualized Return - risk free rate) / Annualized Volatility
    # Annualized return of portfolio minus ETF (use trailing 10 years)
    ann_ret_portfolio = (1 + portfolio_returns.mean()) ** trading_days - 1
    ann_ret_etf = (1 + etf_ret.mean()) ** trading_days - 1

    # Annualized volatility
    ann_vol_portfolio = portfolio_returns.std() * np.sqrt(trading_days)
    ann_vol_etf = etf_ret.std() * np.sqrt(trading_days)

    # Sharpe Ratio of ETF (annualized)
    sharpe_etf = (ann_ret_etf - risk_free_rate) / ann_vol_etf

    # Annualized volatility spread (portfolio vol - ETF vol)
    vol_spread = ann_vol_portfolio - ann_vol_etf

    # Add to table
    comparison_table.loc[etf, 'Correlation vs Portfolio'] = corr
    comparison_table.loc[etf, 'Covariance vs Portfolio'] = cov
    comparison_table.loc[etf, 'Tracking Error (Annualized)'] = tracking_error
    comparison_table.loc[etf, 'Sharpe Ratio'] = sharpe_etf
    comparison_table.loc[etf, 'Annualized Volatility Spread'] = vol_spread

# Reset index for presentation if desired
comparison_table.reset_index(inplace=True)

print(comparison_table)


[*********************100%***********************]  7 of 7 completed
[*********************100%***********************]  3 of 3 completed

  ETF Ticker  Correlation vs Portfolio  Covariance vs Portfolio  \
0        SPY                  0.774157                 0.000182   
1        IWM                  0.785197                 0.000224   
2        DIA                  0.711240                 0.000143   

   Tracking Error (Annualized)  Sharpe Ratio  Annualized Volatility Spread  
0                     0.196876      0.523605                      0.121303  
1                     0.188470      0.191849                      0.074189  
2                     0.219714      0.440199                      0.125775  


In [70]:
# Define tickers
portfolio_tickers = ['GEV', 'TGTX', 'PRVA', 'CEVA', 'GILD', 'KD', 'NRG']
etf_tickers = ['SPY', 'IWM', 'DIA']

# Dates
end_date = datetime.now()
start_date = end_date - timedelta(days=365*10)

# Download data
data_portfolio = yf.download(portfolio_tickers, start=start_date, end=end_date)['Close']
data_etfs = yf.download(etf_tickers, start=start_date, end=end_date)['Close']

# Calculate daily returns
returns_portfolio = data_portfolio.pct_change().dropna()
returns_etfs = data_etfs.pct_change().dropna()

# Equal-weighted portfolio returns
weights = np.repeat(1 / len(portfolio_tickers), len(portfolio_tickers))
portfolio_returns = returns_portfolio.dot(weights)

# Combine all returns in one DataFrame:
#  - Portfolio (named 'Portfolio')
#  - ETFs (SPY, IWM, DIA)
#  - Individual stocks (tickers)

# Rename columns for clarity
returns_etfs.columns = [etf.upper() for etf in returns_etfs.columns]
returns_portfolio.columns = [col.upper() for col in returns_portfolio.columns] if isinstance(returns_portfolio, pd.DataFrame) else []

# Create a DataFrame to hold all
combined_returns = pd.concat([portfolio_returns.rename('Portfolio'), returns_etfs, returns_portfolio], axis=1)

# Calculate correlation matrix
corr_matrix = combined_returns.corr()

print(corr_matrix)

[*********************100%***********************]  7 of 7 completed
[*********************100%***********************]  3 of 3 completed

           Portfolio       DIA       IWM       SPY      CEVA       GEV  \
Portfolio   1.000000  0.711240  0.785197  0.774157  0.732166  0.689515   
DIA         0.711240  1.000000  0.850839  0.951911  0.490496  0.490233   
IWM         0.785197  0.850839  1.000000  0.870153  0.577350  0.546524   
SPY         0.774157  0.951911  0.870153  1.000000  0.602968  0.614287   
CEVA        0.732166  0.490496  0.577350  0.602968  1.000000  0.445366   
GEV         0.689515  0.490233  0.546524  0.614287  0.445366  1.000000   
GILD        0.302157  0.343917  0.239771  0.223551  0.139525 -0.051254   
KD          0.684597  0.497961  0.517013  0.516334  0.475502  0.312705   
NRG         0.712864  0.490487  0.524722  0.599279  0.464021  0.621600   
PRVA        0.434767  0.400525  0.504802  0.324116  0.206910  0.148647   
TGTX        0.503764  0.282762  0.332708  0.263222  0.133124  0.149698   

               GILD        KD       NRG      PRVA      TGTX  
Portfolio  0.302157  0.684597  0.712864  0.434767